In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import numpy as np

In [2]:
def get_pe_ratio(ticker):
    
    driver = webdriver.Chrome()
    driver.get("https://www.wisesheets.io/roe/{}".format(ticker))
    wait = WebDriverWait(driver, 10)

    quarterly_view_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'Toggle_tab__Zahrp')))
    quarterly_view_button.click()

    time.sleep(1) 
    page_source = driver.page_source 

    soup = BeautifulSoup(page_source, 'html.parser')
    table = soup.find('table', class_='styles_stockTable__tLJoD')
    headers = [header.text for header in table.find_all('th')]
    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:  
        cells = row.find_all('td')
        if len(cells) == 3:
            pe_ratio = cells[1].text.strip()
            data.append(pe_ratio)
    driver.quit()
    
    return data

In [3]:
tickers = list(pd.read_csv("data/sp500_close_data.csv").columns[1:])

In [7]:
data = []
for i in tqdm(tickers):
    data.append(get_pe_ratio(i))

df = pd.DataFrame(np.array(data).T, columns=tickers)

100%|██████████| 100/100 [04:55<00:00,  2.96s/it]


In [9]:
df.index = (pd.date_range(start='2014-07-01', end='2024-04-01', freq='Q').strftime('%Y-%m-%d').tolist())[::-1]
df

,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADP,AMAT,AMD,AMGN,...,TXN,UNH,UNP,UPS,V,VRTX,VZ,WFC,WMT,XOM
2024-03-31,31.86%,17.10%,3.16%,6.18%,4.01%,0.86%,25.61%,9.46%,0.22%,-2.25%,...,6.51%,-1.63%,10.48%,6.58%,11.52%,5.93%,4.88%,2.55%,6.28%,4.00%
2023-12-31,45.77%,7.93%,4.13%,7.40%,8.98%,1.30%,20.34%,11.58%,1.19%,12.31%,...,8.11%,6.15%,11.17%,9.27%,12.31%,5.51%,-2.93%,1.86%,6.55%,3.73%
2023-09-30,36.94%,14.70%,3.83%,5.34%,8.89%,1.40%,24.75%,12.26%,0.54%,22.60%,...,10.28%,6.52%,10.91%,5.88%,12.09%,6.27%,4.87%,3.19%,0.57%,4.54%
2023-06-30,32.98%,15.73%,3.70%,7.93%,8.73%,2.44%,22.13%,10.34%,0.05%,20.34%,...,10.80%,6.27%,11.89%,10.40%,10.66%,5.92%,4.88%,2.74%,9.92%,3.96%
2023-03-31,38.87%,1.80%,1.78%,6.41%,8.78%,2.71%,28.24%,11.15%,-0.25%,53.12%,...,11.21%,6.54%,13.09%,9.46%,11.04%,4.85%,5.29%,2.75%,2.31%,5.75%
2022-12-31,52.88%,14.33%,2.82%,8.55%,8.37%,2.63%,27.22%,12.79%,0.04%,44.14%,...,13.46%,5.85%,13.47%,17.45%,11.32%,5.89%,7.22%,1.59%,8.18%,6.54%
2022-09-30,40.89%,24.69%,4.02%,7.53%,7.90%,2.57%,30.23%,13.05%,0.12%,58.66%,...,15.82%,6.74%,16.14%,15.23%,11.07%,7.14%,5.60%,1.88%,-2.49%,10.56%
2022-06-30,33.46%,6.31%,5.53%,8.35%,8.42%,2.04%,19.39%,13.31%,0.81%,54.44%,...,16.26%,6.65%,14.44%,17.49%,9.61%,6.79%,6.04%,1.61%,6.64%,10.07%
2022-03-31,37.11%,27.57%,6.91%,7.95%,9.19%,2.11%,22.14%,13.27%,1.42%,161.14%,...,15.70%,6.60%,13.70%,17.27%,10.13%,6.99%,5.47%,1.96%,2.67%,3.24%
2021-12-31,48.14%,26.25%,5.56%,8.80%,8.33%,0.75%,13.75%,15.07%,12.99%,28.34%,...,16.04%,5.42%,12.08%,21.70%,10.94%,7.62%,5.64%,2.92%,4.28%,5.26%


In [10]:
df.to_csv("historical_roe.csv")